In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
NEO4J_URI=os.getenv('NEO4J_URI')
NEO4J_USERNAME=os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD=os.getenv('NEO4J_PASSWORD')

In [3]:
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEW4J_PASSWORD"]=NEO4J_PASSWORD

In [4]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

C:\Users\PC\AppData\Local\Temp\ipykernel_3216\1556602031.py:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)


In [5]:
## Dataset 

Movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)

FOREACH (director in split(row.director,'|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))

FOREACH (actor in split (row.actors,'|')|
    MERGE (p:Person{name:trim(actor)})
    MERGE (p)-[:ACTED_IN]-> (m))

FOREACH (genre in split(row.genres,'|')|
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))

"""

In [6]:
graph.query(Movie_query)

[]

In [7]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {name: STRING, POB: STRING, yob: INTEGER}
company {name: STRING}
Ai_Engineer {name: STRING, POB: STRING, YOB: INTEGER}
country {name: STRING}
Person {name: STRING, born: INTEGER}
Movie {relased: INTEGER, title: STRING, id: STRING, released: DATE, imdbRating: FLOAT}
User {name: STRING, city: STRING, userID: INTEGER, age: INTEGER}
post {content: STRING, timestamp: DATE_TIME}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)
(:User)-[:POSTED]->(:post)
(:User)-[:LIKES]->(:User)
(:User)-[:FRIEND]->(:User)


In [8]:
from langchain_groq import ChatGroq

groq_api_key=os.getenv('GROQ_API_KEY')

llm=ChatGroq(model='Gemma2-9b-It',groq_api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E27FBCA190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E27E118F10>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
llm.invoke('hi')

AIMessage(content='Hello! 👋 How can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 10, 'total_tokens': 23, 'completion_time': 0.023636364, 'prompt_time': 0.00117212, 'queue_time': 0.26028081, 'total_time': 0.024808484}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9ebec0d4-2455-44b9-9766-02c7f594c2c3-0', usage_metadata={'input_tokens': 10, 'output_tokens': 13, 'total_tokens': 23})

In [11]:
from langchain.chains import GraphCypherQAChain

chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x000001E25DBE38D0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E27FBCA190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E27E118F10>, model_name=

In [16]:
response=chain.invoke({"query":"who was the director of the movie Toy story"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Toy Story" })<-[:DIRECTED]-(p:Person)
RETURN p.name

Full Context:
[{'p.name': 'John Lasseter'}]

> Finished chain.


{'query': 'who was the director of the movie Toy story',
 'result': 'John Lasseter  \n'}

In [21]:
response=chain.invoke({"query":"who was the actors of the movie Toy story"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Toy Story"})<-[:ACTED_IN]-(p:Person) RETURN p.name 

Full Context:
[{'p.name': 'Jim Varney'}, {'p.name': 'Tim Allen'}, {'p.name': 'Tom Hanks'}, {'p.name': 'Don Rickles'}]

> Finished chain.


{'query': 'who was the actors of the movie Toy story',
 'result': 'Jim Varney, Tim Allen, Tom Hanks, Don Rickles were the actors of the movie Toy story. \n'}